# Double Clustering - Estimating Standard Errors in Finance Panel Data sets: Comparing Approaches (Petersen, 2008, RFS) 
------
**Simulation prepared by Jinkyu Kim (Dept. of Finance, Business School, Hanyang Univ.)**

## Motivation

It is well known that OLS standard errors are unbiased when the residuals are independent and identically distributed. When the residuals are correlated across observations, OLS standard errors can be **biased** and either over or **underestimate** the true variability of the coefﬁcient estimates.

There are two general forms of dependence that are most common in ﬁnance applications. They will serve as the basis for the analysis. The residuals of a given ﬁrm may be correlated across years for a given ﬁrm (time-series dependence). I will call this an unobserved **ﬁrm effect** (Wooldridge, 2007). Alternatively, the residuals of a given year may be correlated across different ﬁrms (cross-sectional dependence). I will call this a **time effect**.

Provided standard errors and methods
----------
OLS, White Error, Newey-West, Pooled OLS (same as OLS), Clustered by Firm, Clustered by Time (STATA default), Clustered by Firm and Time (STATA default), Fama and Macbeth

When to use each each standard error?
----------
**FIRM EFFECT**: USE Std. Error **Clustered by FIRM**, or if you sure your firm effect is permanent, FE, RE (I don't provide here, if you need, just search on google) is okay

**TIME EFFECT**: USE **Fama MacBeth**, or if T is sufficient, Std. Error clustered by Time is okay.

**FIRM & TIME EFFECT**: if N,T is sufficient, **Double Clustering**, if not, consider using combination of **Time Dummy + Std. Error Clustered by FIRM**


Simulation
------------
First, generate data. Assume there are 100 firms and 120 data points. 

(120 time points may represent monthly data with the period of 10 yr.)



In [27]:
rm(list=ls())
#Library
library(sandwich); library(plm); library(lmtest);library(dplyr)

Note that gamma and mu is firm effect (time constant) and delta and zeta is time effect (firm constant) and eta and nu is idiosyncratic part.

In [28]:
mydat = data.frame(firm=rep(1:100,each=120),time=rep(1:120, 100), 
                   y=0, x=0, e=0, gamma=0, delta=0, eta=0, mu=0, zeta=0, nu=0)

In [42]:
# Firm Effect : gamma and mu
mydat$gamma = rep(rnorm(0,1,n=100), each=120)
mydat$mu = rep(rnorm(0,1,n=100), each=120)

# Time Effect : delta and zeta
mydat$delta = rep(rnorm(0,1,n=120),100)
mydat$zeta = rep(rnorm(0,1,n=120),100)

# Idiosyncratic : eta and nu
mydat$eta = rnorm(0,1,n=12000)
mydat$nu = rnorm(0,1,n=12000)

# Summation
mydat$e = mydat$gamma+mydat$delta+mydat$eta
mydat$x = mydat$mu+mydat$zeta+mydat$nu

# True Beta
beta = 2
mydat$y = beta*mydat$x+mydat$e


In [43]:
head(mydat); tail(mydat); str(mydat)

firm,time,y,x,e,gamma,delta,eta,mu,zeta,nu
1,1,1.3815607,0.3329437,0.71567334,-0.8937133,-0.4758154,2.08520199,-1.474299,1.10412610,0.70311612
1,2,-2.7766405,-2.3070654,1.83749029,-0.8937133,2.2040380,0.52716560,-1.474299,-0.09608803,-0.73667879
1,3,-4.2428991,-2.1365789,0.03025873,-0.8937133,1.3200122,-0.39604021,-1.474299,0.26068842,-0.92296879
1,4,-1.7546231,-0.7490638,-0.25649562,-0.8937133,0.5671154,0.07010228,-1.474299,1.11317089,-0.38793609
1,5,0.2930051,-0.1552199,0.60344493,-0.8937133,0.8477039,0.64945431,-1.474299,1.40308638,-0.08400775
1,6,-8.1301963,-3.8558375,-0.41852127,-0.8937133,-0.1570221,0.63221411,-1.474299,-1.74591131,-0.63562765


,firm,time,y,x,e,gamma,delta,eta,mu,zeta,nu
11995,100,115,0.3015461,-0.6994800,1.7005062,2.134898,0.2584846,-0.6928762,-1.709656,1.4979568,-0.48778047
11996,100,116,-1.3635671,-3.1648104,4.9660536,2.134898,2.7181466,0.1130092,-1.709656,-1.4716816,0.01652758
11997,100,117,-4.6824022,-3.2066662,1.7309302,2.134898,-0.7471466,0.3431790,-1.709656,-0.7485216,-0.74848822
11998,100,118,5.8581717,1.1416317,3.5749083,2.134898,0.9635764,0.4764341,-1.709656,1.5032722,1.34801587
11999,100,119,-6.4835650,-4.3020060,2.1204470,2.134898,1.7996128,-1.8140636,-1.709656,-0.5357431,-2.05660658
12000,100,120,1.0457676,0.1745256,0.6967164,2.134898,-0.1255213,-1.3126601,-1.709656,0.3442118,1.53997014


'data.frame':	12000 obs. of  11 variables:
 $ firm : int  1 1 1 1 1 1 1 1 1 1 ...
 $ time : int  1 2 3 4 5 6 7 8 9 10 ...
 $ y    : num  1.382 -2.777 -4.243 -1.755 0.293 ...
 $ x    : num  0.333 -2.307 -2.137 -0.749 -0.155 ...
 $ e    : num  0.7157 1.8375 0.0303 -0.2565 0.6034 ...
 $ gamma: num  -0.894 -0.894 -0.894 -0.894 -0.894 ...
 $ delta: num  -0.476 2.204 1.32 0.567 0.848 ...
 $ eta  : num  2.0852 0.5272 -0.396 0.0701 0.6495 ...
 $ mu   : num  -1.47 -1.47 -1.47 -1.47 -1.47 ...
 $ zeta : num  1.1041 -0.0961 0.2607 1.1132 1.4031 ...
 $ nu   : num  0.703 -0.737 -0.923 -0.388 -0.084 ...


In [53]:
model = summary(lm(y~x, data=mydat))


Call:
lm(formula = y ~ x, data = mydat)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.0602 -1.1312 -0.0142  1.1217  6.0397 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.293241   0.015150   19.36   <2e-16 ***
x           1.947419   0.008767  222.13   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 1.659 on 11998 degrees of freedom
Multiple R-squared:  0.8044,	Adjusted R-squared:  0.8044 
F-statistic: 4.934e+04 on 1 and 11998 DF,  p-value: < 2.2e-16


In [97]:
est.b = model$coefficients[2,c("Estimate")]
est.se = model$coefficients[2,c("Std. Error")]
(ols.t = (est.b-beta)/est.se)

[1] -5.997738

In [98]:
vcovDC = function(x, ...){
    vcovHC(x, cluster="group", ...) + 
      vcovHC(x, method=c("arellano"), type=c("sss"),cluster = c("time"), ...) - 
      vcovHC(x, method="white1", ...)
  }

  cluster.double = coeftest(p.ols, vcov = function(x) vcovDC(x)) 

  est.b = cluster.double[2,c("Estimate")]
  est.se = cluster.double[2,c("Std. Error")]                       
                            
  (dc.t = (est.b-beta)/est.se)                        
               

[1] -1.33263

OLS t value is -6 and Double Clustering t value is -1.33

## Let's Functionalize.

In [91]:
tval_generate = function(firmno=100, timept=120, rep=100, truebeta = 2){
    ols.t = white.t = newey.t = fmb.t = cluster.firm.t = cluster.time.t = dc.t = NULL
    
    for (iter in 1:rep){   

        # Data Generation
        mydat = data.frame(firm=rep(1:firmno, each=timept),time=rep(1:timept, firmno), 
                           y=0, x=0, e=0, gamma=0, delta=0, eta=0, mu=0, zeta=0, nu=0)

        # Firm Effect : gamma and mu
        mydat$gamma = rep(rnorm(0,1,n=firmno), each=timept)
        mydat$mu = rep(rnorm(0,1,n=firmno), each=timept)

        # Time Effect : delta and zeta
        mydat$delta = rep(rnorm(0,1,n=timept),firmno)
        mydat$zeta = rep(rnorm(0,1,n=timept),firmno)

        # Idiosyncratic : eta and nu
        mydat$eta = rnorm(0,1,n=firmno*timept)
        mydat$nu = rnorm(0,1,n=firmno*timept)

        # Summation
        mydat$e = mydat$gamma+mydat$delta+mydat$eta
        mydat$x = mydat$mu+mydat$zeta+mydat$nu

        # True Beta
        beta = truebeta
        mydat$y = beta*mydat$x+mydat$e

        # OLS
        ols = lm(y~x, data=mydat)
        est.b = summary(ols)$coefficients[2,c("Estimate")]
        est.se = summary(ols)$coefficients[2,c("Std. Error")]
        ols.t = c(ols.t, (est.b-beta)/est.se)
        
        # White
        white = coeftest(ols, vcov = function(x) vcovHC(x, method="white1", type="HC1"))
        est.b = white[2,c("Estimate")]
        est.se = white[2,c("Std. Error")]                       
        white.t = c(white.t, (est.b-beta)/est.se)                  

        # Newey-West
        newey = coeftest(ols, vcov = NeweyWest(ols))
        est.b = newey[2,c("Estimate")]
        est.se = newey[2,c("Std. Error")]                       
        newey.t = c(newey.t, (est.b-beta)/est.se)

        # Fama - Macbeth
        fmb = pmg(y~x, mydat, index=c("time","firm"))
        FMB = coeftest(fmb)
        est.b = FMB[2,c("Estimate")]
        est.se = FMB[2,c("Std. Error")]
        fmb.t = c(fmb.t, (est.b-beta)/est.se)

        p.ols = plm(y~x, model="pooling", index=c("firm", "time"), data=mydat)
        
        # Cluster by Firm
        cluster.firm = coeftest(p.ols, vcov = function(x) vcovHC(x, cluster="group", type="HC1"))                 
        est.b = cluster.firm[2,c("Estimate")]
        est.se = cluster.firm[2,c("Std. Error")]
        cluster.firm.t = c(cluster.firm.t, (est.b-beta)/est.se)
                                
        # Cluster by Time
        cluster.time = coeftest(p.ols, vcov = function(x) vcovHC(x, method=c("arellano"), type=c("sss"),cluster = c("time")))            
        est.b = cluster.time[2,c("Estimate")]
        est.se = cluster.time[2,c("Std. Error")]
        cluster.time.t = c(cluster.time.t, (est.b-beta)/est.se)
                                
        # Double Cluster
        cluster.double = coeftest(p.ols, vcov = function(x) vcovDC(x)) 
        est.b = cluster.double[2,c("Estimate")]
        est.se = cluster.double[2,c("Std. Error")]                       
        dc.t = c(dc.t, (est.b-beta)/est.se)
        }    
                                  
    # Return
    res = data.frame(ols = ols.t, white = white.t, nw = newey.t, fmb = fmb.t, 
                     firmc = cluster.firm.t, timec = cluster.time.t, dc = dc.t)
    return(res)
}



### Simulation 1000 times
------
I simulate the data of 100 firms and 120 time points with the method of OLS, White, Newey-West, Fama-Macbeth, Firm-Cluster, Time-Cluster, and Double-Cluster 1000 times. So, it will take some time. If you want to see quick results, adjust firmno, timept, and rep variable in the function.

In [92]:
t.val = tval_generate(firmno=100, timept=120, rep=1000, truebeta = 2)
head(t.val);tail(t.val)

ols,white,nw,fmb,firmc,timec,dc
9.1474501,8.916399,3.0959873,12.6189039,2.0043678,2.5132427,1.5918960
-0.4980771,-0.512263,-0.2095261,-2.8372351,-0.1316373,-0.1594814,-0.1035805
1.4769917,1.484961,0.7826039,0.8255710,0.4926386,0.3508894,0.2912561
-5.7281928,-5.776097,-2.7925559,-7.1620859,-1.6826247,-1.5782366,-1.1747245
1.0086357,1.014273,0.3795174,-0.3133026,0.2446348,0.3191522,0.1978263
-5.0213287,-5.126059,-2.3032951,-7.6666184,-1.4189426,-1.7559756,-1.1302130


,ols,white,nw,fmb,firmc,timec,dc
995,6.146726,6.193132,3.0011693,7.352169,1.8296564,1.9215903,1.3565406
996,4.552936,4.588952,1.9161094,11.567818,1.2251179,1.4931046,0.9679903
997,-6.544112,-6.594092,-2.8747517,-6.944292,-1.8000128,-2.3899722,-1.4733581
998,-4.433875,-4.431198,-1.6402493,-5.969178,-1.0687693,-1.3304935,-0.8484058
999,-4.273249,-4.353246,-1.7642116,-2.778387,-1.1127411,-1.4820476,-0.9090866
1000,-1.117527,-1.157403,-0.7022994,-2.422832,-0.4627207,-0.3167893,-0.2683376


### Then I calculate the rejection rate.
------
5% rejection rate for OLS, White, Fama-Macbeth is over 67% (670 times / 1000 times).

1% rejection rate for OLS, white, Fama-Macbeth is over 60% (600 times / 1000 times).

The rejection rate of **double clustering** seems to be appropriate. 5% (50 times / 1000 times) and 1.2% (12 times / 1000 times) each.


In [96]:
(rejection.5percent = sapply(t.val, function(x) length(which(abs(x)>1.96))/10))
(rejection.1percent = sapply(t.val, function(x) length(which(abs(x)>2.52))/10))                           

ols white    nw   fmb firmc timec    dc 
 67.7  67.7  34.2  75.1  14.2  17.1   5.0

ols white    nw   fmb firmc timec    dc 
 60.1  60.4  21.8  68.3   6.0   7.6   1.2

### Results
--------
**I find the rejection rate calculated from all the other methods except Double Clustering absurd.** It implies that we SHOULD USE DOUBLE CLUSTERING when both time-series correlations and cross-sectional correlations are expected.

References
-----------
Petersen (2008, RFS)

Clustering options are exactly same as the STATA. (option specification)